In [ ]:
# Cell 1: Load Libraries and Read Data from Excel File
# Import necessary libraries
import pandas as pd
import numpy as np

# Define the file name (must match the file name uploaded to Colab)
excel_file = 'Manufacturing_Line_Productivity.xlsx'

# Read the four sheets from the Excel file into separate DataFrames
try:
    df_details = pd.read_excel(excel_file, sheet_name='Line productivity')
    df_products = pd.read_excel(excel_file, sheet_name='Products')
    df_factors = pd.read_excel(excel_file, sheet_name='Downtime factors')
    df_downtime = pd.read_excel(excel_file, sheet_name='Line downtime')

    print("All four sheets loaded successfully!")
    print(f"Details shape: {df_details.shape}")

except ValueError as e:
    print(f"Error reading sheets: {e}. Please ensure sheet names are exact.")

All four sheets loaded successfully!
Details shape: (38, 6)


In [ ]:
# Cell 2: Cleaning and Preprocessing Batch Details (df_details) - FINAL ROBUST TIME FIX

import datetime as dt

# Reload df_details to ensure 'Date', 'Start Time', 'End Time' columns are present
# This is necessary if the cell has been run before and these columns were dropped.
df_details = pd.read_excel(excel_file, sheet_name='Line productivity')

# Function to robustly parse time values, handling Excel floats, strings, and datetime.time objects
def parse_time_component(value):
    if pd.isna(value):
        return pd.NaT
    if isinstance(value, (int, float)): # Excel time values are often floats (fraction of a day)
        # Convert fraction of a day to timedelta, then extract the time part
        return (pd.Timestamp('1900-01-01') + pd.to_timedelta(value, unit='D')).time()
    # If it's already a datetime.time object (from read_excel directly)
    if isinstance(value, dt.time):
        return value
    # If it's a pandas Timestamp object, extract its time
    if isinstance(value, pd.Timestamp):
        return value.time()
    # Otherwise, treat as a string and try to parse it
    try:
        # Use pandas to_datetime for string parsing, then get the time
        return pd.to_datetime(str(value), errors='coerce').time()
    except AttributeError: # .time() fails if pd.to_datetime(str(value), errors='coerce') results in NaT
        return pd.NaT
    except Exception: # Catch any other unexpected errors during parsing
        return pd.NaT

# 1. Prepare Product and Factor tables (renaming for merging)
df_factors = df_factors.rename(columns={'Factor': 'Factor ID', 'Description': 'Factor Description', 'Operator Error': 'Is Operator Error'})
df_products = df_products.rename(columns={'Min batch time': 'Min Batch Duration (min)'})
df_factors['Factor ID'] = df_factors['Factor ID'].astype(int)

# 2. Create Start and End Timestamps (Using Timedelta for Robustness and CLEANUP)
# Convert 'Date' column to datetime type
df_details['Date'] = pd.to_datetime(df_details['Date'], format='%Y-%m-%d')

# Apply the robust parsing function to 'Start Time' and 'End Time' columns
df_details['Clean Start Time'] = df_details['Start Time'].apply(parse_time_component)
df_details['Clean End Time'] = df_details['End Time'].apply(parse_time_component)

# Convert the CLEANED Time (datetime.time objects or NaT) to Timedelta.
# astype(str) converts datetime.time objects to 'HH:MM:SS' strings, which pd.to_timedelta can parse.
# If 'Clean Start Time' contains NaT, .astype(str) yields 'NaT', and pd.to_timedelta('NaT', errors='coerce') yields NaT.
df_details['Time Delta Start'] = pd.to_timedelta(df_details['Clean Start Time'].astype(str), errors='coerce')
df_details['Time Delta End'] = pd.to_timedelta(df_details['Clean End Time'].astype(str), errors='coerce')

# Combine the Date (which is at 00:00:00) with the Time Delta
df_details['Start Timestamp'] = df_details['Date'] + df_details['Time Delta Start']
df_details['End Timestamp'] = df_details['Date'] + df_details['Time Delta End']

# 3. Correct for overnight batches (Critical step for time accuracy)
# If End Timestamp is chronologically before Start Timestamp, it means it ended the next calendar day
mask_overnight = df_details['End Timestamp'] < df_details['Start Timestamp']
df_details.loc[mask_overnight, 'End Timestamp'] += pd.Timedelta(days=1)

# 4. Calculate the Actual Batch Duration (in minutes)
df_details['Actual Duration (min)'] = (df_details['End Timestamp'] - df_details['Start Timestamp']).dt.total_seconds() / 60

# Clean up unnecessary original and temporary columns
df_details = df_details.drop(columns=['Date', 'Start Time', 'End Time', 'Clean Start Time', 'Clean End Time', 'Time Delta Start', 'Time Delta End'])
print("Batch Details cleaned and Timestamps/Duration calculated using the robust Timedelta method.")
print(df_details[['Batch', 'Actual Duration (min)']].head())

Batch Details cleaned and Timestamps/Duration calculated using the robust Timedelta method.
    Batch  Actual Duration (min)
0  422111                  135.0
1  422112                  100.0
2  422113                  110.0
3  422114                  100.0
4  422115                   84.0


In [ ]:
# Cell 3: Transforming Downtime Data (df_downtime) from Wide to Long Format

# Reload df_downtime with header=1 to correctly interpret the second row as headers.
# This ensures 'Batch' and factor IDs become proper column names.
df_downtime = pd.read_excel(excel_file, sheet_name='Line downtime', header=1)

# Convert numeric column names (which might be floats like 1.0, 2.0) to integers
# This loop handles cases where column names are ints or floats representing ints.
new_columns = []
for col in df_downtime.columns:
    try:
        # Check if column name is an integer or a float that represents an integer
        if pd.notna(col) and isinstance(col, (int, float)) and float(col).is_integer():
            new_columns.append(int(col))
        else:
            new_columns.append(col)
    except (ValueError, TypeError): # Catch if col is not numeric
        new_columns.append(col)
df_downtime.columns = new_columns


# Identify factor columns. These are now expected to be integer type column names.
factor_columns = [col for col in df_downtime.columns if isinstance(col, int)]

# Melt the DataFrame to convert to Long Format
df_downtime_long = df_downtime.melt(
    id_vars=['Batch'],
    value_vars=factor_columns,
    var_name='Factor ID',
    value_name='Downtime Duration (min)'
)

# Clean: Drop rows where Downtime Duration is NaN (meaning no downtime for that factor)
df_downtime_long = df_downtime_long.dropna(subset=['Downtime Duration (min)'])

# Ensure Factor ID and Duration are correct integer types
df_downtime_long['Factor ID'] = df_downtime_long['Factor ID'].astype(int)
df_downtime_long['Downtime Duration (min)'] = df_downtime_long['Downtime Duration (min)'].astype(int)

print("Downtime records converted to Long format and cleaned.")
print(df_downtime_long.head())

Downtime records converted to Long format and cleaned.
     Batch  Factor ID  Downtime Duration (min)
38  422111          2                       60
39  422112          2                       20
40  422113          2                       50
44  422117          2                       10
57  422130          2                       20


In [ ]:
# Cell 4: Building the Final Data Model (Merging) and Feature Engineering

# 1. Merge Downtime (Long) with Factors
df_model = pd.merge(df_downtime_long, df_factors, on='Factor ID', how='left')

# 2. Merge with Batch Details (using df_details cleaned in Cell 2)
df_model = pd.merge(df_model, df_details, on='Batch', how='left')

# 3. Merge with Product Details
df_model = pd.merge(df_model, df_products, on='Product', how='left')

# Feature Engineering 1: Calculate Total Downtime per Batch
# Grouping by 'Batch' to get the total downtime
df_downtime_summary = df_model.groupby('Batch')['Downtime Duration (min)'].sum().reset_index().rename(
    columns={'Downtime Duration (min)': 'Total Downtime (min)'}
)
df_model = pd.merge(df_model, df_downtime_summary, on='Batch', how='left')

# Feature Engineering 2: Calculate Deviation from Minimum Required Time
df_model['Duration Deviation (min)'] = df_model['Actual Duration (min)'] - df_model['Min Batch Duration (min)']

# Feature Engineering 3: Operator ID Encoding (for ML models)
# Converting Operator names to numerical IDs
df_model['Operator_ID'] = df_model['Operator'].astype('category').cat.codes

# Final Cleaned Dataset (Remove duplicates and select final columns)
df_cleaned_dataset = df_model.drop_duplicates().sort_values(by=['Batch', 'Factor ID']).reset_index(drop=True)

# Select and reorder final columns for clarity and modeling readiness
final_columns = [
    'Batch', 'Product', 'Flavor', 'Size',
    'Operator', 'Operator_ID', 'Start Timestamp', 'End Timestamp',
    'Min Batch Duration (min)', 'Actual Duration (min)', 'Duration Deviation (min)',
    'Total Downtime (min)', 'Factor ID', 'Factor Description',
    'Downtime Duration (min)', 'Is Operator Error'
]

df_cleaned_dataset = df_cleaned_dataset[final_columns]
print("Final Data Model built successfully.")

Final Data Model built successfully.


In [ ]:
# Cell 5: Final Quality Check and Saving Results

# Quality Check: Identify batches where Total Downtime > Actual Duration (Illogical data)
invalid_batches = df_cleaned_dataset[df_cleaned_dataset['Actual Duration (min)'] < df_cleaned_dataset['Total Downtime (min)']].drop_duplicates(subset=['Batch'])

if not invalid_batches.empty:
    print(f"\n!! WARNING: {len(invalid_batches)} batches have Total Downtime > Actual Duration. These require investigation or removal before ML training.")
    print(invalid_batches[['Batch', 'Actual Duration (min)', 'Total Downtime (min)']].to_markdown(index=False))
else:
    print("\nData Quality Check Passed (No batches found with Total Downtime > Actual Duration).")

print(f"\nCleaned Dataset Shape: {df_cleaned_dataset.shape}")
print("\n--- First 5 rows of the Cleaned Dataset ---")
# Print first 5 rows
print(df_cleaned_dataset.head(5).to_markdown(index=False))

# DELIVERABLE: Save the final cleaned dataset
df_cleaned_dataset.to_csv('cleaned_productivity_data.csv', index=False)
print("\nCleaned dataset saved as 'cleaned_productivity_data.csv'.")


Data Quality Check Passed (No batches found with Total Downtime > Actual Duration).

Cleaned Dataset Shape: (61, 16)

--- First 5 rows of the Cleaned Dataset ---
|   Batch | Product   | Flavor     | Size   | Operator   |   Operator_ID | Start Timestamp     | End Timestamp       |   Min Batch Duration (min) |   Actual Duration (min) |   Duration Deviation (min) |   Total Downtime (min) |   Factor ID | Factor Description   |   Downtime Duration (min) | Is Operator Error   |
|--------:|:----------|:-----------|:-------|:-----------|--------------:|:--------------------|:--------------------|---------------------------:|------------------------:|---------------------------:|-----------------------:|------------:|:---------------------|--------------------------:|:--------------------|
|  422111 | OR-600    | Orange     | 600 ml | Mac        |             3 | 2024-08-29 11:50:00 | 2024-08-29 14:05:00 |                         60 |                     135 |                         75 |     